In [4]:
import os
import requests
import shutil
import h5py
import numpy as np
import os
import pandas as pd
import scipy.cluster.hierarchy as shc
import matplotlib.pyplot as plt
import random
import seaborn as sns
%matplotlib inline


In [91]:
#INPUTS:
#  file_list: list of csv filenames containing expression data. Will concatenate these matrices together and compute distances
def preprocess_samples(file_list):
    sample_source = []
    # Merge all of the files into one dataframe
    df_expression = pd.read_csv(os.path.join(data_path,file_list[0]),index_col = 0)
    sample_source.extend(np.repeat(file_list[0],len(df_expression.columns)))
    for filename in file_list[1:]:
        temp = pd.read_csv(os.path.join(data_path,filename),index_col = 0)
        sample_source.extend(np.repeat(filename,len(temp.columns)))
        df_expression = df_expression.merge(temp,on="/meta/genes")
        
    df_expression.columns = pd.Series(df_expression.columns).str.replace("b","").replace("'","")
    df_expression.index = pd.Series(df_expression.index).str.replace("b","").replace("'","")
    return(df_expression,sample_source)    

In [7]:
data_path = "samples"
file_list = ["Oocyte_expression.csv", "Ovary_expression.csv"]
df,sample_source = preprocess_samples(file_list)

In [8]:
filepath = "C:/Users/abraks/Documents/CS 548/ARCH5/human_matrix.h5"

In [10]:
f = h5py.File(filepath, 'r')

Keys: <KeysViewHDF5 ['data', 'info', 'meta']>


In [18]:
print(f["meta"].keys())

<KeysViewHDF5 ['Sample_channel_count', 'Sample_characteristics_ch1', 'Sample_contact_address', 'Sample_contact_city', 'Sample_contact_country', 'Sample_contact_department', 'Sample_contact_email', 'Sample_contact_institute', 'Sample_contact_laboratory', 'Sample_contact_name', 'Sample_contact_phone', 'Sample_contact_zip-postal_code', 'Sample_data_processing', 'Sample_data_row_count', 'Sample_description', 'Sample_extract_protocol_ch1', 'Sample_geo_accession', 'Sample_instrument_model', 'Sample_last_update_date', 'Sample_library_selection', 'Sample_library_source', 'Sample_library_strategy', 'Sample_molecule_ch1', 'Sample_organism_ch1', 'Sample_platform_id', 'Sample_relation', 'Sample_series_id', 'Sample_source_name_ch1', 'Sample_status', 'Sample_submission_date', 'Sample_supplementary_file_1', 'Sample_supplementary_file_2', 'Sample_taxid_ch1', 'Sample_title', 'Sample_type', 'gene_accession', 'gene_chromosome', 'gene_entrezid', 'gene_hgnc', 'gene_name', 'gene_refseqid', 'gene_synonym', '

In [92]:
df_meta = pd.DataFrame({"SampleGeoAccession":df.columns})

In [87]:
full_data = pd.DataFrame({"Description":f[("meta/Sample_description")], 
                          "Characteristics":f[("meta/Sample_characteristics_ch1")],
                          "SampleGeoAccession":f[("meta/Sample_geo_accession")],
                          "Series ID":f[("meta/Sample_series_id")],
                          "Molecule":f["meta/Sample_molecule_ch1"],
                          "Source Name":f["meta/Sample_title"]})
full_data["SampleGeoAccession"] = full_data["SampleGeoAccession"].apply(lambda x : x.decode("utf-8"))

In [88]:
df_meta = df_meta.merge(full_data,on="SampleGeoAccession",how="outer")

In [90]:
df_meta.loc[19,"Characteristics"]

b'age category: advanced maternal ageXx-xXmaternal age: 40Xx-xXtissue: oocyteXx-xXdevelopmental stage'

In [95]:
f.close()